## Different NN models comparison for segmentation task

### Encoders (Feature extraction models):
1. VGG-16
2. ResNET-50
3. ResNET-101
4. SqueezeNet
5. GoogLeNet

### Decoders (Output models)
1. FCN-8
2. Deeplabs

In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

import keras
import keras_applications
from keras import backend as k
from keras.models import Model 
from keras.layers import Conv2DTranspose, Conv2D, Input, Add

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
%matplotlib inline

## Preparing images

In [3]:
def _resize_image(image):
    return cv2.resize(image, (IMAGE_HEIGHT, IMAGE_WIDTH))

def _rgb_to_greyscale(image):
    grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return np.reshape(grey, (IMAGE_HEIGHT, IMAGE_WIDTH, 1))

def read_images_from_path(path, is_mask=False):
    '''
        read images from filepath and return array with them
    '''
    images = []
    for f in os.listdir(path):
        image = cv2.imread(os.path.join(path, f))
        resized_image = _resize_image(image)
        if is_mask:
            resized_image = _rgb_to_greyscale(resized_image)
        images.append(resized_image)
        
    return np.array(images)

In [4]:
IMAGE_WIDTH = IMAGE_HEIGHT = 224

IMAGE_PATH = '../datasets/Unet_Dataset/Files'
MASK_PATH = '../datasets/Unet_Dataset/Masks'

images = read_images_from_path(IMAGE_PATH)
masks = read_images_from_path(MASK_PATH, is_mask=True)

trainX, valX, trainY, valY = train_test_split(images, masks, test_size = 0.1, random_state = 0)

## Models

In [5]:
###########################################################
###################### Encoders ###########################
###########################################################
def _bridge(encoder, filters):
    bridge = Conv2D(filters=filters, kernel_size=(7,7), activation='relu', padding='same', name='conv6')(encoder.output)
    bridge = Conv2D(filters=filters, kernel_size=(1,1), activation='relu', padding='same', name='conv7')(bridge)
    return Model(inputs=encoder.input, outputs=bridge)    

def vgg16(input_shape, weights=None):
    encoder = keras.applications.vgg16.VGG16(weights=weights, include_top=False, input_shape=input_shape)
    return _bridge(encoder, 4096)

def resnet50(input_shape, weights=None):
    encoder = keras.applications.resnet50.ResNet50(weights=weights, include_top=False, input_shape=input_shape)
    return _bridge(encoder, 2048)

#def resnet101_cropped(input_shape, weights=None):
#    return keras_applications.resnet.ResNet101(weights=weights, include_top=False, input_shape=input_shape)


###########################################################
###################### Decoders ###########################
###########################################################
def fcn_8s(encoder, pre_last_pool, pre_pre_last_pool, nb_classes=1):
    fcn8 = Conv2DTranspose(filters=nb_classes, kernel_size=(4,4),strides=(2,2),padding='same',name='fcn8')(encoder.output)
    block4_pool_conv = Conv2D(filters=nb_classes, kernel_size=(1,1), activation='relu', padding='same', name='pre_last_pool_conv')(pre_last_pool.output)
    fcn8_skip_connected = Add(name='fcn8_plus_pool')([fcn8, block4_pool_conv])
    fcn9 = Conv2DTranspose(filters=nb_classes,kernel_size=(4,4),strides=(2,2),padding='same',name='fcn9')(fcn8_skip_connected)
    block3_pool_conv = Conv2D(filters=nb_classes, kernel_size=(1,1), activation='relu', padding='same', name='pre_pre_last_pool_conv')(pre_pre_last_pool.output)    
    fcn9_skip_connected = Add(name='fcn10_plus_pool')([fcn9, block3_pool_conv])
    fcn10 = Conv2DTranspose(filters=nb_classes,kernel_size=(16,16),strides=(8,8),padding='same',name='fcn10',activation='sigmoid')(fcn9_skip_connected)
    return Model(inputs=encoder.input, outputs=fcn10)


In [6]:
vgg16 = vgg16(input_shape=trainX[1].shape)
model = fcn_8s(vgg16, vgg16.get_layer('block4_pool'), vgg16.get_layer('block3_pool'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
_____________________________________

In [7]:
resnet50 = resnet50(input_shape=trainX[1].shape)
resnet50.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

/anaconda3/envs/rzd/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [8]:
#model.compile(optimizer='Adam', loss='binary_crossentropy')

In [9]:
#model.fit(x=trainX, y=trainY, batch_size=10, validation_data=(valX, valY), epochs=200, verbose=1)